In [55]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import sys

## Change the language to English to use spacy

In [56]:
captionlist = {
    1: "god name",
    2: "thank god",
    3: "all deaf hearing arabs",
    4: "peace be upon you",
    5: "today i present to you another programme",
    6: "the subject of the study of arabic sign language",
    7: "today words are sparse in religion",
    8: "also ordinary words",
    9: "no partner of god",
    10: "allah is the greatest"

}

captions = pd.DataFrame.from_dict(captionlist, orient='index')
# Change the index to be in range 1 - 10
captions.index = [i+1 for i in range(10)]
captions

,0
1,god name
2,thank god
3,all deaf hearing arabs
4,peace be upon you
5,today i present to you another programme
6,the subject of the study of arabic sign language
7,today words are sparse in religion
8,also ordinary words
9,no partner of god
10,allah is the greatest


In [57]:
IMG_SIZE = 224
START = 'startdeq'
STOP = 'stopdeq'
MAX_SEQ_LENGTH = len(max(captions[0], key=len).split(" ")) + 2
VOCAB = set(" ".join(captions[0]).split(" ")) | {START, STOP}
VOCAB_SIZE = len(VOCAB)
IMG_SIZE = 224
EMBEDDING_SHAPE = 300
OUTPUT_DIM = 2048
MAX_SEQ_LENGTH, VOCAB_SIZE, VOCAB

(11,
 38,
 {'all',
  'allah',
  'also',
  'another',
  'arabic',
  'arabs',
  'are',
  'be',
  'deaf',
  'god',
  'greatest',
  'hearing',
  'i',
  'in',
  'is',
  'language',
  'name',
  'no',
  'of',
  'ordinary',
  'partner',
  'peace',
  'present',
  'programme',
  'religion',
  'sign',
  'sparse',
  'startdeq',
  'stopdeq',
  'study',
  'subject',
  'thank',
  'the',
  'to',
  'today',
  'upon',
  'words',
  'you'})

In [58]:


directory = '../data/train'

videos = []
images_dir = []
labels = []
df = pd.DataFrame()
# iterate over files in
# that directory
for label in os.listdir(directory):
    f1 = os.path.join(directory, label)
    for video in os.listdir(f1):
        f2 = os.path.join(f1, video)
        for frame in os.listdir(f2):
            videos.append(f2)
            images_dir.append(os.path.join(f2, frame))
            labels.append(int(label))

df['video_name'] = videos
df['image_dir'] = images_dir
df['caption'] = labels
df['caption'] = df['caption'].apply(lambda x: captions[0][x])

df.head()


,video_name,image_dir,caption
0,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,../data/train/0003/01_0003_(10_03_21_21_04_26)...,all deaf hearing arabs
1,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,../data/train/0003/01_0003_(10_03_21_21_04_26)...,all deaf hearing arabs
2,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,../data/train/0003/01_0003_(10_03_21_21_04_26)...,all deaf hearing arabs
3,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,../data/train/0003/01_0003_(10_03_21_21_04_26)...,all deaf hearing arabs
4,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,../data/train/0003/01_0003_(10_03_21_21_04_26)...,all deaf hearing arabs


How many unique videos?

In [59]:
len(np.unique(df['video_name']))

534

In [60]:
df_videos = pd.DataFrame()
videos = df['video_name'].unique()
df_videos = pd.DataFrame(videos, columns=['video_name'])
df_videos['images'] = df_videos['video_name'].apply(lambda x: df[df['video_name'] == x]['image_dir'].values)
df_videos['captions'] = df_videos['video_name'].apply(lambda x: df[df['video_name'] == x]['caption'].values[0])
df_videos

,video_name,images,captions
0,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,[../data/train/0003/01_0003_(10_03_21_21_04_26...,all deaf hearing arabs
1,../data/train/0003/01_0003_(10_03_21_21_05_28)_c,[../data/train/0003/01_0003_(10_03_21_21_05_28...,all deaf hearing arabs
2,../data/train/0003/01_0003_(10_03_21_21_04_36)_c,[../data/train/0003/01_0003_(10_03_21_21_04_36...,all deaf hearing arabs
3,../data/train/0003/01_0003_(10_03_21_21_08_11)_c,[../data/train/0003/01_0003_(10_03_21_21_08_11...,all deaf hearing arabs
4,../data/train/0003/02_0003_(27_02_21_19_40_00)_c,[../data/train/0003/02_0003_(27_02_21_19_40_00...,all deaf hearing arabs
...,...,...,...
529,../data/train/0006/02_0006_(16_02_21_19_09_22)_c,[../data/train/0006/02_0006_(16_02_21_19_09_22...,the subject of the study of arabic sign language
530,../data/train/0006/01_0006_(16_02_21_19_33_14)_c,[../data/train/0006/01_0006_(16_02_21_19_33_14...,the subject of the study of arabic sign language
531,../data/train/0006/02_0006_(16_02_21_19_10_16)_c,[../data/train/0006/02_0006_(16_02_21_19_10_16...,the subject of the study of arabic sign language
532,../data/train/0006/02_0006_(16_02_21_19_05_40)_c,[../data/train/0006/02_0006_(16_02_21_19_05_40...,the subject of the study of arabic sign language


Build CNN to extract features

In [61]:
from keras import applications
def build_feature_extractor():
    InceptionV3_model = applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3), )

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocess_input = keras.applications.inception_v3.preprocess_input(inputs)
    outputs = InceptionV3_model(preprocess_input)
    myModel = keras.Model(inputs, outputs, name="feature_extractor")
    return myModel

feature_extractor = build_feature_extractor()

In [62]:

def image_reader(img_paths):
    images = []
    for path in img_paths:
        image = plt.imread(path)
        image = resize_image(image)
        images.append(image)
    return np.array(images)


def resize_image(image):
    """
    Resize images to a desired shape, does not account for aspect ratio 
    """
    _,_,depth = image.shape
    return np.resize(image, (IMG_SIZE, IMG_SIZE, depth))

    
def encode_video(video_images):
    """
    Takes a video and encode all its frames and return the encoded frames and the captions
    The backbone of the generator as it allows dynamic reading of the data
    """
    videoFeatures = []
    frames = image_reader(video_images)
    features = feature_extractor.predict(frames, verbose=0)
    videoFeatures.append(features.squeeze())
    return np.squeeze(np.array(videoFeatures))

In [63]:
# df_encoded = pd.DataFrame()
# encodings, captions = [], []
# for index, row in df_videos.iterrows():
#     encoding = encode_video(row['images'])
#     encodings.append(encoding)
#     captions.append(row['captions'])

# df_encoded['video'] = df_videos['video_name']
# df_encoded['encoding'] = encodings
# df_encoded['caption'] = captions
# df_encoded.head()

In [64]:
# df_encoded.to_pickle('../data/encoded_train.pkl')
df_encoded = pd.read_pickle('../data/encoded_train.pkl')

In [65]:
df_encoded['caption'] = df_encoded['caption'].apply(lambda x: START + ' ' + x + ' ' + STOP)
df_encoded.head()

,video,encoding,caption
0,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...",startdeq all deaf hearing arabs stopdeq
1,../data/train/0003/01_0003_(10_03_21_21_05_28)_c,"[[0.08136222, 0.055209946, 0.40456566, 0.13459...",startdeq all deaf hearing arabs stopdeq
2,../data/train/0003/01_0003_(10_03_21_21_04_36)_c,"[[0.0, 0.038305435, 0.13834573, 0.17418434, 0....",startdeq all deaf hearing arabs stopdeq
3,../data/train/0003/01_0003_(10_03_21_21_08_11)_c,"[[0.014240551, 0.14180474, 0.3848084, 0.298231...",startdeq all deaf hearing arabs stopdeq
4,../data/train/0003/02_0003_(27_02_21_19_40_00)_c,"[[0.054453805, 0.31206432, 0.49797043, 0.08702...",startdeq all deaf hearing arabs stopdeq


In [66]:
from keras.preprocessing.text import Tokenizer

def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [67]:
captions_tokenized, tokenizer = tokenize(df_encoded['caption'])
df_encoded['caption_tokenized'] = captions_tokenized
df_encoded.head()
df_encoded['encoding'].iloc[0].shape

(80, 2048)

In [68]:
# max_length = max([len(caption) for caption in captions_tokenized])
# df_encoded['caption_tokenized_padded'] = df_encoded['caption_tokenized'].apply(lambda x: keras.utils.pad_sequences([x], maxlen=max_length, padding='post')[0])
# df_encoded['caption_tokenized_padded'].head()

In [69]:
# df_encoded.to_pickle('../data/encoded_train_final.pkl')
def generate_training_samples(row):
    """
    Takes a row of the encoded dataframe and generates the training samples
    """
    # Get the caption
    caption = row['caption_tokenized']
    # Get the encoding
    encoding = row['encoding']
    # Get the video name
    video_name = row['video']
    # Get the length of the caption
    caption_length = len(caption)
    # Get the number of samples
    inputs =[]
    outputs = []
    videos = []
    encodings = []
    
    for i in range(1, caption_length):
        videos.append(video_name)
        encodings.append(encoding)
        inputs.append(caption[:i])
        outputs.append(caption[i])
    return pd.DataFrame({'video': videos, 'encodings': encodings, 'input': inputs, 'output': outputs})

In [70]:
generate_training_samples(df_encoded.iloc[0])

,video,encodings,input,output
0,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...",[1],29
1,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29]",30
2,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29, 30]",31
3,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29, 30, 31]",32
4,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29, 30, 31, 32]",2


In [71]:
df_training = pd.DataFrame()
for index, row in df_encoded.iterrows():
    df_training = pd.concat([df_training, generate_training_samples(row)])

df_training.head()

,video,encodings,input,output
0,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...",[1],29
1,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29]",30
2,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29, 30]",31
3,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29, 30, 31]",32
4,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29, 30, 31, 32]",2


In [72]:
# df_training.to_pickle('../data/training_samples_no_padding.pkl')

In [73]:
df_training['input'] = df_training['input'].apply(lambda x: keras.utils.pad_sequences([x], maxlen=11, padding='post')[0])
df_training['output'] = df_training['output'].apply(lambda x: keras.utils.to_categorical(x, num_classes=len(tokenizer.word_index) + 1))
df_training.head()

,video,encodings,input,output
0,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29, 30, 0, 0, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29, 30, 31, 0, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,../data/train/0003/01_0003_(10_03_21_21_04_26)_c,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29, 30, 31, 32, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [74]:
# df_training.to_pickle('../data/training_samples_padded.pkl')

In [83]:
df_fit2 = df_training[['encodings', 'input', 'output']].copy()
df_fit2

,encodings,input,output
0,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29, 30, 0, 0, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29, 30, 31, 0, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[[0.10615034, 0.07749777, 0.15894692, 0.015735...","[1, 29, 30, 31, 32, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
5,"[[0.071131654, 0.07954844, 0.28291216, 0.15527...","[1, 3, 24, 5, 3, 25, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
6,"[[0.071131654, 0.07954844, 0.28291216, 0.15527...","[1, 3, 24, 5, 3, 25, 5, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,"[[0.071131654, 0.07954844, 0.28291216, 0.15527...","[1, 3, 24, 5, 3, 25, 5, 26, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,"[[0.071131654, 0.07954844, 0.28291216, 0.15527...","[1, 3, 24, 5, 3, 25, 5, 26, 27, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [75]:

# combined_inputs = [[i,j] for i,j in zip( df_training['encodings'].values,df_training['input'].values)]
# df_fit = pd.DataFrame({'input': combined_inputs, 'output': df_training['output'].to_numpy()})
# df_fit.head()

In [76]:
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

build the embedding matrix

In [77]:
import spacy
nlp = spacy.load('en_core_web_lg')
embedding_matrix = np.zeros((VOCAB_SIZE+1, EMBEDDING_SHAPE))

for word in VOCAB:
    index = tokenizer.word_index[word]
    embedding_matrix[index] = nlp(word).vector


In [78]:
# def data_generator(df, batch_size):
#   # x1 - Training data for df
#   # x2 - The caption that goes with each photo
#   # y - The predicted rest of the caption
#   x1, x2, y = [], [], []
#   n=0
#   videos = np.unique(df['video_name']).tolist()
#   x1, x2, y = [],[], []
#   while True:
#     n+=1
#     x1.append(df.iloc[n-1]['encodings'])
#     x2.append(df.iloc[n-1]['input'])
#     y.append(df.iloc[n-1]['outpus'])
#     if n%batch_size == 0:
#         yield ([np.array(x1), np.array(x2)], np.array(y))
#         x1, x2,y = [], [], []


In [137]:
from keras.models import Model
from keras.layers import LSTM, Embedding, add

inputs1 = keras.Input(shape=(80, OUTPUT_DIM))
# fe1 = Dropout(0.5)(inputs1)
_,state_h, state_c = LSTM(32, activation='relu', return_state=True)(inputs1)


inputs2 = keras.Input(shape=(MAX_SEQ_LENGTH,))
se1 = Embedding(VOCAB_SIZE, EMBEDDING_SHAPE, mask_zero=True)(inputs2)
# se2 = Dropout(0.5)(se1)
se3 = LSTM(32)(se1,  initial_state =[state_h, state_c])
outputs = Dense(VOCAB_SIZE+1, activation='softmax')(se3)
caption_model = Model(inputs=[inputs1, inputs2], outputs=outputs)

# caption_model.layers[2].set_weights([embedding_matrix])
# caption_model.layers[2].trainable = False
adamOptimizer = keras.optimizers.Adam(learning_rate = 1e-4)
caption_model.compile(loss='categorical_crossentropy', optimizer=adamOptimizer, metrics=['accuracy'])
caption_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 11)]         0           []                               
                                                                                                  
 input_11 (InputLayer)          [(None, 80, 2048)]   0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 11, 300)      11400       ['input_12[0][0]']               
                                                                                                  
 lstm_4 (LSTM)                  [(None, 32),         266368      ['input_11[0][0]']               
                                 (None, 32),                                                

In [134]:
# def convert_to_numpy(input_list):
#   # Initialize an empty list for storing the converted arrays
#   output_arrays = []
  
#   # Loop through the elements of the input list
#   for element in input_list:
#     # If the element is a list, recursively convert it to a NumPy array
#     if isinstance(element, list):
#       element = convert_to_numpy(element)

#     # Convert the element to a NumPy array
#     element = np.array(element)

#     # Append the element to the list of output arrays
#     output_arrays.append(element)

#   # Return the list of output arrays
#   return output_arrays

# df_fit['input'] = df_fit['input'].apply(lambda x: convert_to_numpy(x))
# df_fit['output'] = df_fit['output'].apply(lambda x: convert_to_numpy(x))

data = df_fit2.to_numpy()
data0 = tf.convert_to_tensor(data[:,0].tolist())
data2 = tf.convert_to_tensor(data[:,1].tolist())
data3 = tf.convert_to_tensor(data[:,2].tolist())
# data = data.tolist()
# dataset = tf.data.Dataset.from_tensor_slices(data)
# get the first axis of the data


In [131]:
# import tensorflow as tf

# class MyDataset(tf.data.Dataset):
#   def __init__(self, data):
#     self.data = data

#   def __len__(self):
#     # Return the length of the dataset
#     return len(self.data)

#   def __getitem__(self, index):
#     # Return the data item at the specified index
#     i,j,k = self.data[index]
#     print(i,j,k)
#     return [i,j], k

# dataset = MyDataset.zip((dataset1, dataset2, dataset3))


In [138]:
caption_model.fit(x=[data0, data2], y = data3, epochs=1, batch_size= 32)

2022-12-06 23:31:39.976750: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-06 23:31:41.436421: W tensorflow/core/common_runtime/forward_type_inference.cc:332] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	while inferring type of node 'cond_40/output/_23'
2022-12-06 23:31:41.446042: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-06 23:31:44.843481: I tensorflow/co

46/94 [=============>................] - ETA: 4:19 - loss: 3.6518 - accuracy: 0.1012